In [ ]:
import pandas as pd


df = pd.read_csv("data/labeling_in_progress_2023_12_28_13_20.csv")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ruptures as rpt

amount_of_plots = 200

for i in range(amount_of_plots):
    norm_prices = np.array([df.at[i, f'norm_price_{j + 1}'] for j in range(25)])
    buy_sell_labels = [df.at[i, f'sep_point_{j + 1}'] for j in range(25)]

    model = "l1"
    algo = rpt.Pelt(model=model, min_size=3, jump=1).fit(norm_prices)
    result = algo.predict(pen=0.4)

    rpt.display(norm_prices, result)

    for (start, end) in zip([0] + result[:-1], result):
        segment = norm_prices[start:end]
        x = np.arange(len(segment))
        coefs = np.polyfit(x, segment, deg=2)  # Выбор степени полинома
        poly = np.poly1d(coefs)
        plt.plot(np.arange(start, end), poly(x), color="red")

    plt.show()


    # Вывод информации о сегментах
    print("Обнаруженные точки изменения:", result)
    print(f"Истинные сегменты: {buy_sell_labels}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import ruptures as rpt

amount_of_plots = 500

for i in range(amount_of_plots):
    norm_prices = np.array([df.at[i, f'norm_price_{j + 1}'] for j in range(25)])
    buy_sell_labels = [df.at[i, f'sep_point_{j + 1}'] for j in range(25)]

    model = "l1"
    algo = rpt.Pelt(model=model, min_size=3, jump=1).fit(norm_prices)
    result = algo.predict(pen=0.4)

    plt.figure(figsize=(17, 7))  # Увеличение размера графика

    # Чередуем цвета фона для сегментов
    colors = ['lightblue', 'lightpink']
    for index, (start, end) in enumerate(zip([0] + result[:-1], result)):
        plt.fill_betweenx(y=[min(norm_prices), max(norm_prices)], x1=start, x2=end, color=colors[index % 2], alpha=0.3)

    # Рисуем исходный временной ряд с точками
    plt.plot(norm_prices, color='blue', marker='o')  # Голубой цвет линии с точками
    # Чередуем цвета фона для сегментов
    colors = ['lightblue', 'lightpink']
    for index, (start, end) in enumerate(zip([0] + result[:-1], result)):
        plt.fill_between(np.arange(start, end), norm_prices[start:end], color=colors[index % 2], alpha=0.2)

    # Рисуем регрессии для каждого сегмента
    for (start, end) in zip([0] + result[:-1], result):
        segment = norm_prices[start:end]
        x = np.arange(start, end)

        # Linear regression
        if len(segment) > 1: 
            reg = LinearRegression().fit(x.reshape(-1, 1), segment)
            plt.plot(x, reg.predict(x.reshape(-1, 1)), color="orange", label='Линейная регрессия')

        # Poly 2
        if len(segment) > 3:  # Убедитесь, что для полинома 3-ей степени есть достаточно точек
            coefs = np.polyfit(x - start, segment, deg=2)
            poly = np.poly1d(coefs)
            plt.plot(x, poly(x - start), color="green")

        # Poly 3
        if len(segment) > 3:  # Убедитесь, что для полинома 3-ей степени есть достаточно точек
            coefs = np.polyfit(x - start, segment, deg=3)
            poly = np.poly1d(coefs)
            plt.plot(x, poly(x - start), color="purple", label='Полином 3-й степени')


        # Добавляем текст в углу
        plt.text(0.05, 0.98, 'Линейная регрессия', color='orange', transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')
        plt.text(0.19, 0.98, 'Полином 2-й степени', color='green', transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')
        plt.text(0.33, 0.98, 'Полином 3-й степени', color='purple', transform=plt.gca().transAxes, fontsize=10, verticalalignment='top')


    plt.show()
